In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Lookthrough - Option on Euro-Bund Futures

Attributes
----------
valuation
transactions
instruments
recipes
options
futures
bonds
securitised portfolios
exposure
lookthrough
"""

toggle_code("Hide docstring")

The Euro-Bund is a futures contract assigned by the Federal Republic of Germany, and traded on the Eurex Exchange.

In this notebook, we demonstrate the booking of Options on Euro-Bund contracts via instrument references, as well as how we can net the option's exposure to the bond price against the exposure due to any units of the bond that are directly held.

This demonstrates a few technical features:
- Specify underlyings of derivatives via reference to a ClientInternalId or LusidInstrumentId ticker
- Nested lookthrough via ReferenceInstrument
- Net exposure to underlyings of derivatives

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
import json
import pytz
import uuid
from datetime import datetime, timedelta
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from flatten_json import flatten

import os
import pandas as pd
import math

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

#Load LUSID API Components
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
reference_portfolio_api = api_factory.build(lusid.api.ReferencePortfolioApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

# Set Global Scope
global_scope = "OptionOnFutureOnBond"
pricing_date = datetime(2023, 1, 17, tzinfo=pytz.utc)

print ('LUSID Environment Initialised')
print ('LUSID API Version :', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.6.11013.0


In [3]:
def upsert_instrument(name, client_id, defn):
    response = instruments_api.upsert_instruments(
        request_body={
            client_id: models.InstrumentDefinition(
                name=name,
                identifiers={
                    "ClientInternal": models.InstrumentIdValue(value=client_id)
                },
                definition=defn,
            )
        },
        scope = global_scope
    )
    luid = response.values[client_id].lusid_instrument_id
    print(f"{name} = " + luid)
    return luid

In [4]:
# define a bond
def create_flow_conventions(currency, payment_frequency):
    return models.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention="None",
        day_count_convention="ActAct",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=0,
        reset_days=0)
    
def create_bond(
    currency,
    payment_frequency,
    start_date,
    maturity_date,
    principal,
    coupon_rate,
    bond_identifier,
    identifier_type,
):
    flow_conventions = create_flow_conventions(currency, payment_frequency)
    
    return models.Bond(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=currency,
        principal=principal,
        coupon_rate=coupon_rate,
        flow_conventions=flow_conventions,
        identifiers={identifier_type: bond_identifier},
        instrument_type="Bond",
        calculation_type="Standard",
    )

currency = "EUR"
payment_frequency = "1Y"
start_date = datetime(2021, 2, 15, tzinfo=pytz.utc)
maturity_date = datetime(2032, 2, 15, tzinfo=pytz.utc)
principal = 100_000
coupon_rate = 6.0
bond_identifier = "DE0001102580"
identifier_type = "Isin"

bond_definition = create_bond(
    currency,
    payment_frequency,
    start_date,
    maturity_date,
    principal,
    coupon_rate,
    bond_identifier,
    identifier_type,
)

bond_luid = upsert_instrument("DBR 0 02/15/32", "MyBond", bond_definition)

DBR 0 02/15/32 = LUID_00004A81


In [5]:
# Define a EURO-BUND future specifying the bond (via its ClientInternalId) as its underlying
# This ignores some subtleties such as cheapest-to-deliver and conversion factors

ref_to_bond = models.ReferenceInstrument(
        instrument_type = "ReferenceInstrument",
        scope = global_scope,
        instrument_id_type = "ClientInternal",
        instrument_id = "MyBond"
    )

future_definition = models.Future(
    instrument_type = "Future",
    start_date = datetime(2022, 6, 8, tzinfo=pytz.utc),
    maturity_date = datetime(2023, 3, 10, tzinfo=pytz.utc),
    identifiers = {"Isin": "DE000C6YTCM4"},
    contract_details = models.FuturesContractDetails(dom_ccy = "EUR",
                                                    asset_class = "InterestRates",
                                                    contract_code = "FGBL",
                                                    contract_month = "H", # March
                                                    contract_size = 100_000,
                                                    exchange_code = "EUX"),
    contracts = 1,
    ref_spot_price = 138.27,
    underlying = ref_to_bond
)

future_luid = upsert_instrument("RXH3 Comdty Future", "MyFuture", future_definition)

RXH3 Comdty Future = LUID_00004A82


In [6]:
# Define an exchange-traded option specifying the future (via its LusidInstrumentId) as its underlying
option_definition = models.ExchangeTradedOption(
    instrument_type = "ExchangeTradedOption",
    start_date = pricing_date,
    contract_details = models.ExchangeTradedOptionContractDetails(
        dom_ccy = "EUR",
        option_code = "OGBL",
        underlying_code = "FGBL",
        strike = 138.0,
        contract_size = 100_000,
        country = "Germany",
        delivery_type = "Physical",
        description = "some description",
        exchange_code = "OGBL",
        exercise_date = datetime(2023, 2, 24, tzinfo=pytz.utc),
        exercise_type = "American",
        option_type = "Call",
        underlying = models.ReferenceInstrument(
            instrument_type = "ReferenceInstrument",
            scope = global_scope,
            instrument_id_type = "LusidInstrumentId",
            instrument_id = future_luid
        )
    ),
    contracts = 1,
    ref_spot_price = 2.00
)

option_luid = upsert_instrument("RXH3C 138 Call Option", "MyOption", option_definition)

RXH3C 138 Call Option = LUID_00004A83


In [7]:
# Upsert equity quotes into LUSID
def upsert_quote(ticker, ticker_type, price, scale_factor = 1):
    instrument_quotes = {
    ticker: models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=ticker,
                    instrument_id_type=ticker_type,
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=pricing_date.isoformat(),
            ),
            metric_value=models.MetricValue(value=price, unit="EUR"),   
            scale_factor = scale_factor
        )
    }
    
    response = quotes_api.upsert_quotes(scope=global_scope, request_body=instrument_quotes) 

def upsert_hacky_quote(ticker, ticker_type, price, quote_type = "Price", scale_factor = 1):
    instrument_quotes = {
    ticker: models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=ticker,
                    instrument_id_type=ticker_type,
                    quote_type=quote_type,
                    field="mid",
                ),
                effective_at=pricing_date.isoformat(),
            ),
            metric_value=models.MetricValue(value=price, unit="EUR"),
            scale_factor = scale_factor
        )
    }
    
    response = quotes_api.upsert_quotes(scope="hacky_scope", request_body=instrument_quotes) 

upsert_quote(option_luid, "LusidInstrumentId", 2.00) # simple-static ETO base dependency
upsert_quote(future_luid, "LusidInstrumentId", 138.27) # future price
upsert_quote(bond_luid, "LusidInstrumentId", 82.7885, scale_factor = 100) # bond price

upsert_hacky_quote("FGBL", "LusidInstrumentId", 2.00) # simple-static ETO extra dependency
upsert_hacky_quote("Delta/OGBL", "LusidInstrumentId", 0.5300, "Delta") # SS ETO extra dependency (Bloomberg: 53.00)
print("Quotes upserted!")

Quotes upserted!


In [8]:
# Place the option in a portfolio
portfolio_code = "OptionPortfolio"

try:
    create_portfolio = transaction_portfolio_api.create_portfolio(
        scope=global_scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="EUR",
            created=pricing_date,
            sub_holding_keys=[],
            instrument_scopes=[global_scope]
        ),
    )
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])
    
# Set trade variables
trade_date = pricing_date
settle_days = 0

def create_transaction_request(transaction_id, luid, units):
    return models.TransactionRequest(
    transaction_id=transaction_id,
    type="StockIn",
    instrument_identifiers={"Instrument/default/LusidInstrumentId": luid},
    transaction_date=trade_date.isoformat(),
    settlement_date=(trade_date + timedelta(days=settle_days)).isoformat(),
    units=units,
    transaction_price=models.TransactionPrice(price=1,type="Price"),
    total_consideration=models.CurrencyAndAmount(amount=1,currency="EUR"),
    exchange_rate=1,
    transaction_currency="EUR"
)

option_txn = create_transaction_request("TXN001", option_luid, 1)

response = transaction_portfolio_api.upsert_transactions(scope=global_scope,
                                                    code=portfolio_code,
                                                    transaction_request=[option_txn])

print(f"Transactions successfully updated at time: {response.version.as_at_date}")

Could not create a portfolio with id 'OptionPortfolio' because it already exists in scope 'OptionOnFutureOnBond'.
Transactions successfully updated at time: 2023-01-18 11:52:24.170137+00:00


We need to instruct the configuration recipe via a model rule to expand the exchange-traded option into its underlying future, and another mode rule to expand the future into its underlying bond. Here, we choose the expansion methodology 'Sum', and in the pricing options specify exposure as quantity to conserve. This essentially means that the option will display as an equivalent number of units of the bond, such that the exposure of the option is conserved.

In [9]:
def UpsertRecipe(recipe_code, conserved_quantity):
    if conserved_quantity is not None:
        expansion_options = models.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions")
    else:
        expansion_options = None
    model_rules=[]
    model_rules.append(models.VendorModelRule(
        supplier="Lusid",
        model_name="SimpleStatic",
        instrument_type="ExchangeTradedOption",
        model_options=expansion_options))
    model_rules.append(models.VendorModelRule(
        supplier="Lusid",
        model_name="SimpleStatic",
        instrument_type="Future",
        model_options=expansion_options))
    model_rules.append(models.VendorModelRule(
        supplier="Lusid",
        model_name="SimpleStatic",
        instrument_type="Bond"))
    
    configuration_recipe = models.ConfigurationRecipe(
        scope=global_scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.RIC.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                ),
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                ),
                models.MarketDataKeyRule(
                    key="Quote.LusidInstrumentId.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                ),
                models.MarketDataKeyRule(
                    key="Quote.LusidInstrumentId.*",
                    supplier="Lusid",
                    data_scope="hacky_scope", # SHOULD BE REMOVED
                    quote_type="Price",
                    field="mid",
                ),
                models.MarketDataKeyRule(
                    key="Quote.LusidInstrumentId.*",
                    supplier="Lusid",
                    data_scope="hacky_scope", # SHOULD BE REMOVED
                    quote_type="Delta",
                    field="mid",
                ),
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=global_scope,
                attempt_to_infer_missing_fx=True             
            ),
        ),
        pricing=models.PricingContext(
            model_rules=model_rules,
            options=models.PricingOptions(
                conserved_quantity_for_lookthrough_expansion=conserved_quantity)
        ),    
    )

    upsert_configuration_recipe_response = (
        configuration_recipe_api.upsert_configuration_recipe(
            upsert_recipe_request=models.UpsertRecipeRequest(
                configuration_recipe=configuration_recipe
            )
        )
    )
    
UpsertRecipe("ExpandByExposureRecipe", "Exposure")
UpsertRecipe("DontExpandRecipe", None)

print("Recipes upserted!")

Recipes upserted!


In [10]:
# Create a valuation function allowing users to specify the look-through recipe
def get_daily_val(date, portfolio_code, recipe_code, group_by = None):
    group_by = [group_by] if group_by else []
    metricsList = []
    columnsToRename={
            f"Instrument/default/Name": "Instrument Name",
            f"Instrument/default/LusidInstrumentId": "LUID",
            f"Sum(Valuation/PvInReportCcy)": "PV (Reporting Ccy)",
            f"Sum(Valuation/ExposureInReportCcy)": "Exposure (Reporting Ccy)",
            f"Holding/default/FundLineage": "Fund Lineage",
            f"Sum(Holding/default/Units)": "Implied Holding Units"
        }
    
    metricsList.extend([
            models.AggregateSpec(f"Instrument/default/Name", "Value"),
            models.AggregateSpec(f"Instrument/default/LusidInstrumentId", "Value"),
            models.AggregateSpec(f"Holding/default/FundLineage", "Value"),
            models.AggregateSpec(f"Holding/default/Units", "Sum"),
            models.AggregateSpec(f"Valuation/UltimateUnderlying", "Value"),
            models.AggregateSpec(f"Valuation/PvInReportCcy", "Sum"),        
            models.AggregateSpec(f"Valuation/ExposureInReportCcy", "Sum"),
        ])
          
    # Build and run valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=global_scope, code=recipe_code),
        metrics=metricsList,
        group_by=group_by,
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=global_scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date.isoformat()),
        report_currency = "EUR"
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data
    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns=columnsToRename,
        inplace=True,
    )

    return vals_df

In [11]:
# when pricing the option, we can see that a bond is the ultimate underlying
get_daily_val(pricing_date, "OptionPortfolio", "DontExpandRecipe")

,Instrument Name,LUID,Fund Lineage,Implied Holding Units,Valuation/UltimateUnderlying,PV (Reporting Ccy),Exposure (Reporting Ccy)
0,RXH3C 138 Call Option,LUID_00004A83,OptionPortfolio,1.00,{ ClientInternal : MyBond },"200,000.00","106,000.00"


In [12]:
# perform lookthrough on the option while preserving exposure 
# as with the no-lookthrough case, this tells us our exposure to the underlying
# it also tells us the number of units of the underlying that that exposure implies
get_daily_val(pricing_date, "OptionPortfolio", "ExpandByExposureRecipe")

,Instrument Name,LUID,Fund Lineage,Implied Holding Units,Valuation/UltimateUnderlying,PV (Reporting Ccy),Exposure (Reporting Ccy)
0,DBR 0 02/15/32,LUID_00004A81,OptionPortfolio/LUID_00004A82/LUID_00004A81,1.28,{ ClientInternal : MyBond },"813,185.77","106,000.00"


In [13]:
# using the implied units from the previous API call, 
# we can hedge our exposure by booking an opposing number of units of the underlying
bond_txn = create_transaction_request("TXN002", bond_luid, -1.28)
bond_txn_response = transaction_portfolio_api.upsert_transactions(scope=global_scope,
                                                    code=portfolio_code,
                                                    transaction_request=[bond_txn])

print("Underlying bond added to portfolio!")

Underlying bond added to portfolio!


In [14]:
# at that point, we can re-price our portfolio and group by the underlying to show a net exposure near zero
get_daily_val(pricing_date, "OptionPortfolio", "ExpandByExposureRecipe", "Instrument/default/LusidInstrumentId")

,Instrument Name,LUID,Fund Lineage,Implied Holding Units,Valuation/UltimateUnderlying,PV (Reporting Ccy),Exposure (Reporting Ccy)
0,DBR 0 02/15/32,LUID_00004A81,None,0.00,{ ClientInternal : MyBond },235.67,30.72


In [15]:
cancel_bond_txn = transaction_portfolio_api.cancel_transactions(scope=global_scope,
                                                    code=portfolio_code,
                                                    transaction_ids=["TXN002"])

print("Underlying bond removed from portfolio!")

Underlying bond removed from portfolio!
